In [1]:
from pathlib import Path
import pandas as pd
import string
raw = Path("transcript_raw.txt").read_text()

preprocess raw: delete newlines and reporters' names, delete excess whitespace and all punctuation:

In [2]:
raw = raw.replace("\nVekarić:", " ").replace("\nB92:", " ").replace("\n", " ").lower()
for l in string.punctuation:
    raw = raw.replace(l, "")
raw = " ".join(raw.split())

In [3]:
df = pd.read_csv("003_all_models_transcriptions.csv")
df.head()

,model,transcription
0,classla/wav2vec2-xls-r-parlaspeech-hr-lm,ukoliko ste i teevziejsupravo završene svakako...
1,classla/wav2vec2-xls-r-parlaspeech-hr,ukoliko ste ti televzije bkoji su upravo zavr...
2,classla/wav2vec2-large-slavic-parlaspeech-hr,<pad>u<pad>kol<pad>i<pad>ko s<pad>te<pad> <pad...
3,classla/wav2vec2-large-slavic-parlaspeech-hr-lm,ukoliko ste 92 koji su upravo svakako vam je d...


In [4]:
def generate_differ_content(pred:str) -> str:
    """Generate comparatively pretty diff from input predictions
    against global raw.

    Args:
        pred (str): Predictions to be compared

    Returns:
        str: Newline delimited diff.
    """
    from difflib import Differ
    differ_content = ""
    for diff in list(Differ().compare(raw.split(), pred.split()))[:500]:
        differ_content += diff + "\n"
    return differ_content

In [6]:
for i, row in df.iterrows():
    modelname = row["model"]
    pred = row["transcription"].replace("<pad>", "")
    from datasets import load_metric

    wer = load_metric("wer")
    cer = load_metric("cer")

    c = cer.compute(predictions = [pred], references = [raw])
    w = wer.compute(predictions = [pred], references = [raw])

    content = generate_differ_content(pred=pred)
    fname = f"""003_{modelname.replace("/", "--")}.txt"""
    p = Path("./diffs") / fname
    header = f"""Model: {modelname},\ncer: {c}, \nwer: {w}\n\n\n"""
    p.write_text(header + content)
